# OAI2-Abfrage mit ChatGPT

In diesem Beispiel zeigen wir, wie Sie die OAI2-Schnittstelle der Deutschen Nationalbibliothek (DNB) mit Python programmgesteuert abfragen können, um Metadaten zu erhalten. Das Ziel ist es, die ersten 10 Datensätze aus dem Set `dnb:digitalisate-oa:projekt29` abzurufen, sie in einem Dataframe darzustellen und als CSV-Datei herunterzuladen.

## Voraussetzungen

Um den Code auszuführen, benötigen Sie die folgenden Python-Bibliotheken:

- requests
- xml.etree.ElementTree
- pandas
- IPython.display

Sie können diese Bibliotheken über `pip` installieren, falls sie nicht bereits auf Ihrem System installiert sind.

## Schritt-für-Schritt-Anleitung

1. Importieren Sie die erforderlichen Bibliotheken.

2. Definieren Sie die URL zur OAI2-Schnittstelle der DNB.

3. Definieren Sie eine Funktion, um Metadaten über die OAI2-Schnittstelle abzurufen.

4. Geben Sie den Set-Spezifikator an, von dem Sie die Datensätze abrufen möchten.

5. Legen Sie fest, wie viele Datensätze abgerufen werden sollen.

6. Rufen Sie die Metadaten über die OAI2-Schnittstelle ab.

7. Erstellen Sie eine leere Liste, um die extrahierten Datensätze zu speichern.

8. Extrahieren Sie die Metadaten der ersten 10 Datensätze und fügen Sie sie zur Liste hinzu.

9. Erstellen Sie einen DataFrame aus den extrahierten Datensätzen.

10. Füllen Sie leere Felder im DataFrame mit "N.N.".

11. Erstellen Sie eine formatierte HTML-Tabelle mit fetten Spaltenüberschriften.

12. Zeigen Sie die Tabelle an.

13. Speichern Sie den DataFrame als CSV-Datei.

---

Fügen Sie den Code und den Beschreibungstext in separate Zellen in Ihrem Jupyter Notebook ein, um das Beispiel auszuführen. Sie können den Code in der ersten Zelle ausführen, indem Sie die Zelle markieren und die Ausführungstaste drücken (z. B. die Tastenkombination "Shift" + "Enter"). Der Dataframe wird angezeigt, und die CSV-Datei wird im gleichen Verzeichnis wie das Notebook erstellt.

Bitte stellen Sie sicher, dass Sie die erforderlichen Bibliotheken installiert haben, bevor Sie den Code ausführen.


In [1]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from IPython.display import display, HTML

# OAI-URL
oai_url = 'https://services.dnb.de/oai2/repository'

# Funktion zur Abfrage von Metadaten
def get_records(set_spec):
    params = {
        'verb': 'ListRecords',
        'set': set_spec,
        'metadataPrefix': 'oai_dc'
    }
    response = requests.get(oai_url, params=params)
    return response.content

# Set-Spezifikator
set_spec = 'dnb:digitalisate-oa:projekt29'

# Anzahl der Datensätze, die heruntergeladen werden sollen
num_records = 10

# Metadaten abfragen
records_response = get_records(set_spec)
records_xml = ET.fromstring(records_response)
records = records_xml.findall('.//{http://www.openarchives.org/OAI/2.0/}record')

# Liste zum Speichern der Datensätze erstellen
data = []

# Datensätze extrahieren
count = 0
for record_element in records:
    if count >= num_records:
        break

    metadata = record_element.find('.//{http://www.openarchives.org/OAI/2.0/}metadata')
    oai_dc = metadata.find('.//{http://www.openarchives.org/OAI/2.0/oai_dc/}dc')

    # Metadaten extrahieren
    title = oai_dc.find('.//{http://purl.org/dc/elements/1.1/}title').text
    creator_element = oai_dc.find('.//{http://purl.org/dc/elements/1.1/}creator')
    creator = creator_element.text if creator_element is not None else "N.N."
    date = oai_dc.find('.//{http://purl.org/dc/elements/1.1/}date').text

    # Datensatz zur Liste hinzufügen
    data.append({'Title': title, 'Creator': creator, 'Date': date})

    count += 1

# DataFrame erstellen
df = pd.DataFrame(data)

# Leere Felder mit "N.N." füllen
df.fillna("N.N.", inplace=True)

# HTML-Tabelle mit fetten Spaltenüberschriften erstellen
html_table = df.style.set_table_styles(
    [{'selector': 'th', 'props': [('font-weight', 'bold')]}]
).render()

# Tabelle anzeigen
display(HTML(html_table))

# DataFrame als CSV-Datei speichern
csv_filename = 'dnb_records.csv'
df.to_csv(csv_filename, index=False)
print(f'Der DataFrame wurde als "{csv_filename}" erfolgreich gespeichert.')


/tmp/ipykernel_154/3552658692.py:62: FutureWarning: this method is deprecated in favour of `Styler.to_html()`
  ).render()


,Title,Creator,Date
0,Porträt Jan Janszoon Oomkens (1779 - 1844) / unbezeichnet,Börsenverein der Deutschen Buchhändler / Bibliothek,2019
1,"Bildnis Tycho Brahe / Bez. u.M.: L. Buchhorn sc.; darunter Verlegeradresse: Zwickau, b. d. Gebr. Schumann","Buchhorn, Ludwig [Künstler] [Stecher]",2018
2,"Porträt Carl Gottlob Oertel : Kupferstich, koloriert / unbez.",N.N.,2018
3,Porträt Johannes Schöner (1477 - 1547) : Holzschnitt / unbez.,"Schnellboltz, Gabriel [Drucker]",2018
4,Porträt Michel Joseph de Montgolfier (1740 - 1810) : Heliogravüre? / Bez. u.l.: Boissieu del.; u.r.: Phot. Dumont; unter der Bildlegende: H. LAUNETTE & CIE. ÉDITEURS | Imp. G. Couty,"Boissieu, Claude V. de [Künstler]",2018
5,Porträt Robert Julien van Maldeghem (1810 - 1893) : Holzstich / unbez.,N.N.,2018
6,Porträt Evarist van Maldeghem (1808 - 1875) : Holzstich / Bez. u.l.: Monogramm ICR (?),N.N.,2018
7,"Porträt Gotthold Ephraim Lessing (1729 - 1781) : Stahlstich / Bez. u.l.: A. Weger sc. Leipzig; u.M.: Verlag v. Eduard Volkening, Leipzig.","Weger, August [Stecher]",2018
8,Porträt Förster (?) : Autotypie / Bez. u.r. im Oval in Spiegelschrift: unleserlich,N.N.,2018
9,Porträt Johann Christoph (1563-1637) : Kupferstich / Bez. u.l.: Kager invent:; u.r.: Kilian ...?,"Kager, Johann Matthias [Künstler]",2018


Der DataFrame wurde als "dnb_records.csv" erfolgreich gespeichert.
